In [ ]:
### SETUP

import os, sys

import pandas as pd
pd.options.mode.chained_assignment = None

import warnings
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning)

code_dir = os.getcwd() + '/code/'
data_dir = os.getcwd() + '/data/'
macro_dir = data_dir + '/macro/'

sys.path.append(code_dir)
import fred_data as fd

In [ ]:
### MACRO DATA MAPPING

fred_api_key = 'YOUR_FRED_API_KEY'

macro_as_of_date = '2/8/2023'
gdp_as_of_date = '3/1/2023'

fred_macro_data_lbls = {
    'BBKMGDP': 'gdp',
    'INDPRO': 'ip',
    'CPIAUCSL': 'cpi',
    'UNRATE': 'ur',
    'EMRATIO': 'epr',
    'DGS1': 'gs1',
    'DGS2': 'gs2',
}

local_macro_data_lbls = {
    'crb': 'crb',
    'ebp': 'ebp',
    'wxr': 'wxr',
    'mpi_jk': 'mpi_jk',
    'mpi_jk_pm': 'mpi_jk_pm',
    'mpi_mar': 'mpi_mar',
    'mps_bs': 'mps_bs',
    'mpso_bs': 'mpso_bs',
}

macro_data_lbls = fred_macro_data_lbls | local_macro_data_lbls

In [ ]:
### SVAR-IV MACRO DATA SAVING

date_series = pd.date_range(start = '1992-01-01', end = '2020-02-01', freq = 'MS', name = 'date')
macro_df = pd.DataFrame(date_series)

macro_df['yyyymm'] = 100 * macro_df['date'].dt.year + macro_df['date'].dt.month
macro_df['y'] = macro_df['date'].dt.year
macro_df['m'] = macro_df['date'].dt.month

for lbl in macro_data_lbls:
    if lbl in fred_macro_data_lbls:
        if lbl in ['BBKMGDP', 'INDPRO', 'CPIAUCSL']:
            if lbl == 'BBKMGDP':
                series_df = fd.fred_macro_df(fred_api_key, lbl, gdp_as_of_date, macro_data_lbls[lbl], True)
            else:
                series_df = fd.fred_macro_df(fred_api_key, lbl, macro_as_of_date, macro_data_lbls[lbl], True)
        elif 'DGS' in lbl:
            series_df = fd.fred_macro_df(fred_api_key, lbl, None, macro_data_lbls[lbl], False)
        else:
            series_df = fd.fred_macro_df(fred_api_key, lbl, macro_as_of_date, macro_data_lbls[lbl], False)
    else:
        if lbl == 'crb':
            series_df = fd.csv_to_df(macro_dir, lbl, True)
        else:
            series_df = fd.csv_to_df(macro_dir, lbl, False)
    
    macro_df = macro_df.merge(series_df, on = 'yyyymm', how = 'left')

macro_df.to_csv(os.path.dirname(os.getcwd()) + '/MATLAB/data/macro_data.csv', index = False)